# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### To-Do List
* export images
* add comments


In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
# import cities data from WeatherPy.ipynb file output.
cities_filepath = os.path.join("output_data", "cities.csv")

city_data_df = pd.read_csv(cities_filepath)

city_data_df.head()

,City ID,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,kushmurun,KZ,98,1632243372,67,52.4503,64.6311,40.98,3.67
1,1,saint george,US,0,1632243323,19,37.1041,-113.5841,74.43,1.99
2,2,kodiak,US,0,1632243372,51,57.7900,-152.4072,41.36,25.81
3,3,port alfred,ZA,98,1632243373,69,-33.5906,26.8910,59.18,5.44
4,4,bluff,NZ,35,1632243373,73,-46.6000,168.3333,44.80,16.02


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
# configure gmaps object with api key
gmaps.configure(api_key = g_key)

# create locations data frame
locations = city_data_df[['Lat', 'Lng']].astype(float)

# assign weights  to the cities based on the humidity level.
weights = city_data_df['Humidity'].astype(float)

# create gmps figure
fig = gmaps.figure(zoom_level=2, center=(0,0), layout={'width': '1050px', 'height':'700px'})

# create and add heat map layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='700px', width='1050px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
'''
filter city data to only include cities with the following criteria:

max temp between 70 and 80
cloudiness less than  or equal to 25%
wind speed less than or equal to 5mph
humidity less than or equal to 50%
'''

hotel_df = city_data_df[(city_data_df['Max Temp'] <= 80) & (city_data_df['Max Temp'] > 70) & (city_data_df['Cloudiness']<=25) & (city_data_df['Wind Speed']<=5) & (city_data_df['Humidity'] <=50)]

hotel_df


,City ID,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
1,1,saint george,US,0,1632243323,19,37.1041,-113.5841,74.43,1.99
292,292,barcelos,PT,0,1632243507,35,41.5388,-8.6151,78.85,3.11
299,299,komsomolskiy,UZ,0,1632243419,32,40.4272,71.7189,70.52,0.00
415,415,hami,CN,18,1632243492,24,42.8000,93.4500,71.02,4.32


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
# Create new empty column and rename it to the appropriate title
hotel_df = hotel_df.assign(Hotel_Name = '').rename(columns= {'Hotel_Name': 'Hotel Name'})
hotel_df


,City ID,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,1,saint george,US,0,1632243323,19,37.1041,-113.5841,74.43,1.99,
292,292,barcelos,PT,0,1632243507,35,41.5388,-8.6151,78.85,3.11,
299,299,komsomolskiy,UZ,0,1632243419,32,40.4272,71.7189,70.52,0.00,
415,415,hami,CN,18,1632243492,24,42.8000,93.4500,71.02,4.32,


In [45]:
# base url for google places api
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set parameters for api call
params = {
    'key': g_key,
    'radius': 5000,
    'keyword': 'hotel',
}

# iterrate through each city in the filtered data frame
for index, row in hotel_df.iterrows():
    
    params['location'] = f"{row['Lat']},{row['Lng']}"    
    
    # google places api call
    hotel_response = requests.get(base_url, params).json()
    
    # append in the hotel name request
    hotel_df.loc[index, 'Hotel Name'] = hotel_response['results'][0]['name']


hotel_df


,City ID,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,1,saint george,US,0,1632243323,19,37.1041,-113.5841,74.43,1.99,St. George Inn & Suites
292,292,barcelos,PT,0,1632243507,35,41.5388,-8.6151,78.85,3.11,Hotel Restaurante Bagoeira
299,299,komsomolskiy,UZ,0,1632243419,32,40.4272,71.7189,70.52,0.00,TANTANA HOTEL FERGANA
415,415,hami,CN,18,1632243492,24,42.8000,93.4500,71.02,4.32,Tianma Hotel


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations=locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(hotel_marker)


fig

Figure(layout=FigureLayout(height='700px', width='1050px'))